### Подсчет у таблицы target_class параметры по данным (Merged and cleaned) mdm таблицы
Подсчет у таблицы target_class параметры по данным (Merged and cleaned) mdm таблицы  
Входные данные data/interm/merge_cleaned.pkl из merge_and_clean.ipynb  
  
Подсчитывает по данным федресурса  

|    | feature                                | Наименование                                                                                                                         |
|---:|:---------------------------------------|:-------------------------------------------------------------------------------------------------------------------------------------|
|  0 | no_contracts_total                     | Количество договоров (всех закрытых и действующих), заключенных до даты принятия решения по сделке                                   |
|  1 | no_contracts_active                    | Количество активных (действующих) договоров на дату принятия решения по сделке                                                       |
|  2 | no_transport_total                     | Количество договоров только с транспортными средствами (всех закрытых и действующих), заключенных до даты принятия решения по сделке |
|  3 | no_transp_active                       | Количество активных (действующих) договоров только с транспортными средствами на дату принятия решения по сделке                     |
|  4 | no_contracts_closed                    | Количество закрытых договоров до даты принятия решения по сделке                                                                     |
|  5 | reason_завершение                      | Количество закрытых договоров до даты принятия решения по сделке по причине завершения договора                                      |
|  6 | reason_замена/перенайм/переуступка     | Количество закрытых договоров до даты принятия решения по сделке по причине замена/перенайм/переуступка                              |
|  7 | reason_не указана                      | Количество закрытых договоров до даты принятия решения по сделке по неуказанной причине                                              |
|  8 | reason_определена цессия               | Количество закрытых договоров до даты принятия решения по сделке по причине цессии                                                   |
|  9 | reason_отмена                          | Количество закрытых договоров до даты принятия решения по сделке по причине отмены                                                   |
| 10 | reason_расторгнут                      | Количество закрытых договоров до даты принятия решения по сделке по причине расторжения                                              |
| 11 | reason_страховой случай                | Количество закрытых договоров до даты принятия решения по сделке по причине страхового случая                                        |
| 12 | reason_страховой случай (гибель/тотал) | Количество закрытых договоров до даты принятия решения по сделке по причине страхового случая (гибель/тотал)                         |
| 13 | bad_reasons_closed                     | Количество договоров c признаками плохого завершения до даты принятия решения по сделке                                              |
| 14 | good_reasons_closed                    | Количество договоров c признаками хорошего завершения до даты принятия решения по сделке                                             |
| 15 | not_determined_reasons_closed          | Количество договоров c отсутствием признаков хорошего или плохого завершения до даты принятия решения по сделке                      |
| 16 | mean_bad_reasons                       | Среднее количество закрытых договоров с отрицательным исходом до даты принятия решения по сделке                                     |
| 17 | mean_good_reasons                      | Среднее количество закрытых договоров с положительным исходом до даты принятия решения по сделке                                     |
| 18 | mean_not_determined_reasons            | Среднее количество закрытых договоров с неопределенным исходом до даты принятия решения по сделке                                    |
| 19 | no_creditors                           | Количество различных лизингодателей                                                                                                  |
| 20 | max_duration                           | Максимальная длительность закрытых договоров до даты принятия решения по сделке                                                      |
| 21 | min_duration                           | Минимальная длительность закрытых договоров до даты принятия решения по сделке                                                       |
| 22 | mean_duration                          | Средняя длительность закрытых договоров до даты принятия решения по сделке                                                           |
| 23 | contract_date_delta_first              | Разница в днях между самым первым заключенным договором в mdm до даты принятия решения по сделке                                     |
| 24 | contract_date_delta_last               | Разница в днях между последним заключенным договором в mdm до даты принятия решения                                                  |
| 25 | end_date_last                          | Разница в днях между датой самого последнего завершения в mdm до даты принятия решения                                               |
| 26 | end_date_first                         | Разница в днях между датой самого первого завершения в mdm до даты принятия решения                                                  |
  
Присоединяет рассчитанные фичи к данным для target_class и записываем в data/interm/target_features.pkl файл для дальнейшего анализа  


In [1]:
import pandas as pd

In [2]:
pre = '../../' # путь до главной директории проекта

%run ../init.ipynb #подгрузить нужные библиотеки
%run ../params.ipynb #параметры как логин пароль к БИ, параметры расчёта ( в нашем случае их нет)
%run ../init_mdm.ipynb
%run ../preprocessing/preprocessing_target.ipynb
%run ../ML_functions.ipynb
%run ../auxiliary/plots.ipynb

In [3]:
# сюда таблицы будем кидать, для отчетов
name = pre + "data/interm/auxilary/feature_engineering.xlsx"
writer = ExcelWriter(name)

In [4]:
name = pre + "data/interm/merge_cleaned.pkl"
df_mdm = pd.read_pickle(name, compression={'method':'zip'})

print("Размер считанных данных фед ресурса mdm- {}".format(df_mdm.shape))
# df_mdm.head(1)

Размер считанных данных фед ресурса mdm- (1412787, 28)


In [5]:
name = pre + "data/interm/external_pkl/target_dataset.pkl"
target_class = pd.read_pickle(name, compression={'method':'zip'})

print(len(target_class))
# target_class.head(1)

85798


In [6]:
name = pre + "data/interm/external_pkl/mdl_app.pkl"
mdl_app = pd.read_pickle(name, compression={'method':'zip'})

mdl_app.drop_duplicates(subset='PL', inplace=True)

print(len(mdl_app))
# mdl_app.head(1)

250658


In [7]:
target = pd.merge(target_class, mdl_app, left_on='Приложение', right_on='PL', how='left')
target['found'] = target['PL'].notnull()

target[~target['found']].to_excel(writer, sheet_name='not found in mdl_app', index=False)

target.drop(columns=['PL', 'found'], inplace=True)

# target['O_NEW_TOTALFINANCINGSUM'] = target['O_NEW_TOTALFINANCINGSUM'].fillna(0)

Check_length(len(target_class), len(target)) 
# target.head()

In [8]:
df = pd.merge(target, df_mdm, how='left', left_on='ИНН контрагента', right_on='new_lessee_inn')
#  убираем все строки с датой публикации больше принятия решения, но оставим те, к которым присоединится не получилось 
mask = (df['new_kreditmessage_date']>df['Дата принятия решения по сделке'])
raw_m_data = df[~mask].copy()

# raw_m_data.head(1)

## 1. посчитаем, количество договоров на дату публикации

In [9]:
contacts_total = raw_m_data.groupby('Приложение')['origin_by_message'].aggregate('nunique').to_frame().reset_index().rename(columns={'origin_by_message':'no_contracts_total'})
# contacts_total.head()

## 2. посчитаем, количество договоров на дату публикации (только транспортных средств)

In [10]:
transport_only = raw_m_data[(raw_m_data['new_leasingsubject_code'].notnull()) & (raw_m_data['new_leasingsubject_code'].str.startswith('0106'))]
contacts_transp_total = transport_only.groupby('Приложение')['origin_by_message'].aggregate('nunique').to_frame()\
                                        .reset_index().rename(columns={'origin_by_message':'no_transport_total'})

## 3. посчитаем количество активных договоров

для этого оставим последнюю запись по договам из mdm к нашим приложениям  и там посмотрим, завершен ли там договор или нет

In [11]:
last = raw_m_data.sort_values(by=['Приложение', 'origin_by_message', 'new_kreditmessage_date']).drop_duplicates(subset=['Приложение', 'origin_by_message'], keep='last')

In [12]:
"""stop_primary - предварительно вычисленная дата окончания, там где явно задано окончание, то  его указываем, там где не указано, то по договору лизинга"""
mask = (last['cession_date'].notnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'cession_date']

mask = (last['new_terminationleasing_date'].notnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'new_terminationleasing_date']

mask = (last['stop_primary'].isnull())
last.loc[mask, 'stop_primary'] = last.loc[mask, 'new_leasing_enddate']

mask = (last['stop_primary'].isnull())
mask_stop = (last['new_kreditmessage_type'] == 'StopFinancialLeaseContract')
last.loc[mask & mask_stop, 'stop_primary'] = last.loc[mask & mask_stop, 'new_kreditmessage_date']

# get_contract_by_deal(last, 'АЛ429999').sort_values(by=['origin_by_message', 'new_kreditmessage_date']) \
#                     [['Приложение', 'Дата принятия решения по сделке', 'new_kreditmessage_date', 'new_kreditmessage_type', 
#                     'new_leasing_startdate',	'new_leasing_enddate', 
#                     'new_terminationleasing_date', 'new_terminationleasing_reason', 'new_contract_status', 'stop_primary']]

In [13]:
"""
уберем те, уже закрытые на момент принятия решения, будем считать только активные договоры

"""
mask = (last['Дата принятия решения по сделке']>last['stop_primary'])
raw_active = last[~mask].copy()

# get_contract_by_deal(raw_active, 'АЛ429999').sort_values(by=['origin_by_message', 'new_kreditmessage_date']) \
#                     [['Приложение', 'Дата принятия решения по сделке', 'new_kreditmessage_date', 'new_kreditmessage_type', 
#                     'new_leasing_startdate',	'new_leasing_enddate', 
#                     'new_terminationleasing_date', 'new_terminationleasing_reason', 'new_contract_status', 'stop_primary']]

In [14]:
active = raw_active.groupby('Приложение')['origin_by_message'].aggregate('nunique').to_frame().reset_index().rename(columns={'origin_by_message':'no_contracts_active'})
# active.head()

### 4. Теперь считаем только активные по транспортным средствам

In [15]:
transport_only_active = raw_active[(raw_active['new_leasingsubject_code'].notnull()) & (raw_active['new_leasingsubject_code'].str.startswith('0106'))]
transp_active = transport_only_active.groupby('Приложение')['origin_by_message'].aggregate('nunique').to_frame()\
                            .reset_index().rename(columns={'origin_by_message':'no_transp_active'})
# transp_active.head()

### 5. Расчет фичей:
- no_contracts_closed -  количество завершенных договоров
- количества причин завершения у нашего лизингополучателя:

In [16]:
"""
уберем все активные на момент принятия решения, будем считать только закрытые договоры

"""
mask = (last['Дата принятия решения по сделке']>=last['stop_primary'])

raw_closed = last[mask].copy()

mask = (raw_closed['reason']=='')
raw_closed.loc[mask, 'reason'] = 'не указана'

# get_contract_by_deal(raw_closed, 'АЛ429999').sort_values(by=['origin_by_message', 'new_kreditmessage_date']) \
#                     [['Приложение', 'Номер договора (кратко)', 'Дата принятия решения по сделке', 'new_kreditmessage_date', 'new_kreditmessage_type', 
#                     'new_leasingsubject_code',
#                     'new_leasing_startdate',	'new_leasing_enddate', 
#                     'new_terminationleasing_date', 'new_terminationleasing_reason', 'new_contract_status', 'stop_primary']]

In [17]:
closed = raw_closed.groupby('Приложение')['origin_by_message'].aggregate('nunique').to_frame().reset_index().rename(columns={'origin_by_message':'no_contracts_closed'})
# closed.head()

In [18]:
dummies = pd.get_dummies(raw_closed["reason"], prefix="reason")
col_reasons = dummies.columns.to_list()
raw_closed_reasons = raw_closed.join(dummies)
closed_reasons = raw_closed_reasons.groupby('Приложение')[col_reasons].aggregate('sum').reset_index()
# closed_reasons.head()

bad_reasons_closed - плохое завершение предыдущих договоров  
good_reasons_closed - хорошее завершение предыдущих договоров  
not_determined_reasons_closed - неопределенное завершение предыдущих договоров  

In [19]:
bad = ['reason_гибель/тотал', 'reason_расторгнут', 'reason_страховой случай (гибель/тотал)']
good = ['reason_завершение']
not_determined = ['reason_замена/перенайм/переуступка', 'reason_не указана', 'reason_определена цессия', 'reason_отмена', 'reason_страховой случай']

In [20]:
closed_reasons['bad_reasons_closed'] = closed_reasons[bad].sum(axis=1)
closed_reasons['good_reasons_closed'] = closed_reasons[good].sum(axis=1)
closed_reasons['not_determined_reasons_closed'] = closed_reasons[not_determined].sum(axis=1)

# closed_reasons.head(1)

In [21]:
# небольшая проверка, что все закрытые договоры были разнесены на 3 подгруппы
closed_w_reasons = closed_reasons.copy()
closed_w_reasons = closed_w_reasons.fillna(0)

closed_w_reasons['closed'] = closed_w_reasons[['bad_reasons_closed',	'good_reasons_closed',	'not_determined_reasons_closed']].sum(axis=1)
closed_w_reasons = pd.merge(closed, closed_w_reasons , how = 'outer', on='Приложение')
Check_length(len(closed_w_reasons), len(closed))

tmp = check_fields(closed_w_reasons,'closed', 'no_contracts_closed' )


closed_w_reasons.drop(columns='closed', inplace=True)

In [22]:
mask = (closed_w_reasons['no_contracts_closed']==0)

closed_w_reasons.loc[mask, 'mean_bad_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_bad_reasons'] = closed_w_reasons.loc[~mask, 'bad_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

closed_w_reasons.loc[mask, 'mean_good_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_good_reasons'] = closed_w_reasons.loc[~mask, 'good_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

closed_w_reasons.loc[mask, 'mean_not_determined_reasons'] = 0
closed_w_reasons.loc[~mask, 'mean_not_determined_reasons'] = closed_w_reasons.loc[~mask, 'not_determined_reasons_closed']/closed_w_reasons.loc[~mask, 'no_contracts_closed']

# closed_w_reasons.head(1)

### 6. Длительность закрытых договоров

In [23]:
raw_closed['Длительность'] = raw_closed['stop_primary'] - raw_closed['new_leasingcontract_date']
mask = (raw_closed['Длительность']<pd.Timedelta(days=0))
raw_closed.loc[mask, 'Длительность'] = pd.Timedelta(days=0)

duration = pd.pivot_table(raw_closed, index='Приложение',  values='Длительность',  aggfunc={'min', 'mean', 'max'})\
    .rename(columns={'max':'duration_max', 'mean':'duration_mean', 'min':'duration_min'})

for col in duration.columns:
    if col == 'Приложение':
        continue
    duration[col] = duration[col].round('D')
duration.reset_index(inplace=True)
# duration.head(3)

### 7. Разница между датой принятия решения и датой первого и последнего договора в фед ресурсе

In [24]:
# из-за единичных случаев, когда сообщение опубликовано раньше даты контракта, мы их удалим
last['contract_date_delta'] = last['Дата принятия решения по сделке'] - last['new_leasingcontract_date']
tmp = last[last['contract_date_delta']>=pd.Timedelta(days=0)]
contract_timedelta = pd.pivot_table(tmp, index='Приложение',  values='contract_date_delta', aggfunc={'min', 'max'})\
    .rename(columns={'max':'contract_date_delta_first', 'min':'contract_date_delta_last'}).reset_index()

contract_timedelta['contract_date_delta_last'] = contract_timedelta['contract_date_delta_last'].round('D')
contract_timedelta['contract_date_delta_first'] = contract_timedelta['contract_date_delta_first'].round('D')
# contract_timedelta.head()

In [25]:
contract_timedelta.describe()

,contract_date_delta_first,contract_date_delta_last
count,58457,58457
mean,872 days 01:13:56.984450096,229 days 17:37:54.595001452
std,578 days 07:43:23.450542168,279 days 11:05:46.611282972
min,2 days 00:00:00,1 days 00:00:00
25%,371 days 00:00:00,43 days 00:00:00
50%,819 days 00:00:00,121 days 00:00:00
75%,1304 days 00:00:00,308 days 00:00:00
max,7337 days 00:00:00,2827 days 00:00:00


### 8. Разница между датой принятия решения и датой закрытия договора в фед ресурсе

In [26]:
raw_closed['contract_date_delta'] = raw_closed['Дата принятия решения по сделке'] - raw_closed['stop_primary']

end_date_timedelta = pd.pivot_table(raw_closed, index='Приложение',  values='contract_date_delta', aggfunc={'min', 'max'})\
    .rename(columns={'max':'end_date_first', 'min':'end_date_last'}).reset_index()

end_date_timedelta['end_date_last'] = end_date_timedelta['end_date_last'].round('D')
end_date_timedelta['end_date_first'] = end_date_timedelta['end_date_first'].round('D')
# end_date_timedelta.head()

In [27]:
end_date_timedelta.describe()

,end_date_first,end_date_last
count,33992,33992
mean,662 days 09:44:56.822781832,206 days 12:48:40.498940930
std,469 days 06:34:48.937539736,238 days 07:34:40.389350356
min,0 days 00:00:00,0 days 00:00:00
25%,274 days 00:00:00,42 days 18:00:00
50%,586 days 00:00:00,122 days 00:00:00
75%,979 days 00:00:00,283 days 00:00:00
max,2765 days 00:00:00,2107 days 00:00:00


### 9. Количество различных лизингодателей

In [28]:
last['new_creditor_inn'] = last['new_creditor_inn'].fillna("")
creditors = last.groupby('Приложение')['new_creditor_inn'].aggregate('nunique').to_frame().reset_index().rename(columns={'new_creditor_inn':'no_creditors'})
# creditors.head()

In [29]:
creditors.describe()

,no_creditors
count,62551.000000
mean,2.369251
std,2.008457
min,1.000000
25%,1.000000
50%,2.000000
75%,3.000000
max,24.000000


# присоединим к данным для target_class и записываем в pkl файл

In [30]:
# присоединим к данным для тестирования
tmp = pd.merge(target, active, how = 'left', on='Приложение')
tmp = pd.merge(tmp, transp_active , how = 'left', on='Приложение')


tmp = pd.merge(tmp, contacts_total, how = 'left', on='Приложение')
tmp = pd.merge(tmp, contacts_transp_total , how = 'left', on='Приложение')
tmp = pd.merge(tmp, closed_w_reasons , how = 'left', on='Приложение')

tmp = pd.merge(tmp, creditors , how = 'left', on='Приложение')

tmp = pd.merge(tmp, duration , how = 'left', on='Приложение')
tmp = pd.merge(tmp, contract_timedelta , how = 'left', on='Приложение')
target_features = pd.merge(tmp, end_date_timedelta , how = 'left', on='Приложение')


Check_length(len(target_features), len(target_class))
list_of_features = ['no_contracts_active', 'no_transp_active', 'no_contracts_total', 
                    'no_transport_total', 'no_contracts_closed', 'bad_reasons_closed', 
                    'good_reasons_closed', 'not_determined_reasons_closed', 
                    'mean_bad_reasons','mean_good_reasons', 'mean_not_determined_reasons', 
                    'no_creditors']

for col in list_of_features:
    target_features[col] = target_features[col].fillna(0)

for col in col_reasons:
    target_features[col] = target_features[col].fillna(0)

# get_contract_by_deal(target_features, 'АЛ520677')

## Статистика по фичам

In [31]:
pd.DataFrame(allFeaturesProperties).T.reset_index().rename(columns={'full_name':'Наименование'})

,index,type,Наименование,short_name,title,for_closed
0,no_contracts_total,int,Количество договоров (всех закрытых и действую...,Количество договоров (всех закрытых и действую...,Количество договоров<br>(всех закрытых и дейст...,False
1,no_contracts_active,int,Количество активных (действующих) договоров на...,Количество активных (действующих) договоров,Количество активных (действующих) договоров,False
2,no_transport_total,int,Количество договоров только с транспортными ср...,Количество договоров только с транспортными ср...,Количество договоров только с транспортными ср...,False
3,no_transp_active,int,Количество активных (действующих) договоров то...,Количество активных (действующих) договоров то...,Количество активных (действующих) договоров<br...,False
4,no_contracts_closed,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров,Количество закрытых договоров,False
5,reason_завершение,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров по причине завер...,Количество закрытых договоров <br>по причине з...,True
6,reason_замена/перенайм/переуступка,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров по причине замен...,Количество закрытых договоров<br>по причине за...,True
7,reason_не указана,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров по неуказанной п...,Количество закрытых договоров<br> по неуказанн...,True
8,reason_определена цессия,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров по причине цессии,Количество закрытых договоров<br> по причине ц...,True
9,reason_отмена,int,Количество закрытых договоров до даты принятия...,Количество закрытых договоров по причине отмены,Количество закрытых договоров<br> по причине о...,True


In [32]:
df_names_full = pd.DataFrame(allFeaturesProperties).T.reset_index().rename(columns={'full_name':'Наименование', 'index':'feature'})
# print(df_names_full.to_markdown())
df_names_full[['feature','Наименование']].to_excel(writer, sheet_name='full_feature_names')

In [33]:
describe = target_features.describe().T

null_stat = pd.DataFrame(round(target_features.isnull().sum()/len(target_features)*100, 2), 
                        columns=['Процент null значений'])
stat = describe.join(null_stat).reset_index().rename(columns={'index':'feature'})

tmp = pd.DataFrame(allFeaturesProperties).T
stat = pd.merge(stat, tmp[['short_name']], left_on='feature', right_index=True, how='left').rename(columns = {'short_name':'Наименование'})

for row in ['duration_max', 'duration_mean', 'duration_min',
            'contract_date_delta_first', 'contract_date_delta_last',
            'end_date_first', 'end_date_last' ]:
    mask = (stat['feature']==row)
    stat.loc[mask, 'min'] = stat.loc[mask, 'min'].apply(strfdelta, fmt='{D}days')
    stat.loc[mask,'max'] = stat.loc[mask,'max'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'mean'] = stat.loc[mask,'mean'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'std'] = stat.loc[mask,'std'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'25%'] = stat.loc[mask,'25%'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'50%'] = stat.loc[mask,'50%'].apply(strfdelta, fmt='{Y} years  {D}days')
    stat.loc[mask,'75%'] = stat.loc[mask,'75%'].apply(strfdelta, fmt='{Y} years  {D}days')


stat.to_excel(writer, sheet_name='feature_stat')

stat

,feature,count,mean,std,min,25%,50%,75%,max,Процент null значений,Наименование
0,Метка фрод дефолта,85798.0,0.079571,0.270629,0.0,0.0,0.0,0.0,1.0,0.00,NaN
1,Метка кред дефолта,85798.0,0.04429,0.20574,0.0,0.0,0.0,0.0,1.0,0.00,NaN
2,O_NEW_TOTALFINANCINGSUM,85743.0,19606765.93558,39527206.036861,0.0,2628136.46,6947722.33,21075748.24,749576289.72,0.06,NaN
3,no_contracts_active,85798.0,8.558917,25.729633,0.0,0.0,1.0,6.0,674.0,0.00,Количество активных (действующих) договоров
4,no_transp_active,85798.0,6.788375,20.337542,0.0,0.0,1.0,5.0,594.0,0.00,Количество активных (действующих) договоров то...
5,no_contracts_total,85798.0,12.537495,35.227339,0.0,0.0,2.0,9.0,824.0,0.00,Количество договоров (всех закрытых и действую...
6,no_transport_total,85798.0,9.786557,27.978356,0.0,0.0,2.0,7.0,645.0,0.00,Количество договоров только с транспортными ср...
7,no_contracts_closed,85798.0,3.978578,13.122241,0.0,0.0,0.0,2.0,276.0,0.00,Количество закрытых договоров
8,reason_гибель/тотал,85798.0,0.001375,0.037373,0.0,0.0,0.0,0.0,2.0,0.00,Количество закрытых договоров по причине гибел...
9,reason_завершение,85798.0,1.368622,6.09267,0.0,0.0,0.0,0.0,141.0,0.00,Количество закрытых договоров по причине завер...


In [34]:
# сохраняем tmp таблицы
if len(writer.sheets):
    writer.save()

In [35]:
target_features.to_pickle(pre+"data/interm/target_features.pkl", compression={'method':'zip'})